In [177]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string
from sklearn.cluster import KMeans
import time
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import random
from itertools import combinations


tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

#device = torch.device('cuda:0' if torch.cuda.is_available() else 0)
#model.to(device)

def make_summarization(sentences, tokenizer, model): # 문자열 생성 요약
    if(len(sentences) < 4): return [max(sentences, key=lambda x:len(x))]
    split = []
    for i in range(len(sentences)//8):
        split.append(sentences[:8])
        sentences = sentences[8:]

    for i in range(len(split)):
        if(len(sentences) == 0): break
        split[i].append(sentences.pop())
    
    if(len(sentences) != 0): split.append(sentences)
    
    split_sum = []
    for sentences in split:
        text = '.\n'.join(sentences)
        start = time.time()
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
        #input_ids = input_ids.to(device)

        summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=256, min_length=10,  eos_token_id=1)
        print(f"{time.time()-start:.4f} sec")
        sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        print(sum_result)
        split_sum.append(sum_result)
        print(len(split), len(split_sum))
        print('-----------------------------------------------')
    
    if(len(split_sum)==1):
        return [split_sum[0]]
      
    return split_sum


def summarize_topic(document_df, topic_num, tokenizer, model): # df와 topic num을 넣으면 해당 topic num을 요약
    sentences = []
    numbers = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
        numbers.append(eval(t)[0])
    result = make_summarization(sentences, tokenizer, model)
    avg = sum(numbers)/len(numbers)
    return (avg, result)


def summarize_first_sentences(processed_sentences, tokenizer, model): # 문쟈열을 k-means로 토픽 별 분류(첫 문장)
    clusternum = 1
    document_df = get_clustered_df(processed_sentences, clusternum)
    sum_result = []
    for c in range(clusternum):
        temp = get_topic_sentences(document_df, c)
        summ = summarize_topic(document_df, c, tokenizer, model)
        sum_result.append(summ)
        print('===================================================')
    
    first_result = (sum_result[0][0], [min(sum_result[0][1], key=lambda x:len(x))])
    print(first_result)
    return [first_result]
    

def summarize_topk_sentences(processed_sentences, tokenizer, model): # 문쟈열을 k-means로 토픽 별 분류
    clusternum = max(len(processed_sentences)//7, 1)
    document_df = get_clustered_df(processed_sentences, clusternum)
    sum_result = []
    for c in range(clusternum):
        temp = get_topic_sentences(document_df, c)
        print('---------------------------------------------------')
        summ = summarize_topic(document_df, c, tokenizer, model)
        sum_result.append(summ)
        print(summ)
        print('***************************************************')
        
    return sorted(sum_result, key= lambda x: x[0])


def get_clustered_df(sentences, clusternum):
    print(clusternum)
    
    document_df = pd.DataFrame()
    document_df['opinion_text'] = [str(t) for t in sentences]
    
    if(len(sentences) < 2):
        document_df['cluster_label'] = 0
        print('len document df', len(document_df))
        return document_df.sort_values(by=['cluster_label'])
    
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

    lemmar = WordNetLemmatizer()
    
    # 토큰화한 각 단어들의 원형들을 리스트로 담아서 반환
    def LemTokens(tokens):
        return [lemmar.lemmatize(token) for token in tokens]

    # 텍스트를 Input으로 넣어서 토큰화시키고 토큰화된 단어들의 원형들을 리스트로 담아 반환
    def LemNormalize(text):
        return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
    
    tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                            ngram_range=(1,2),
                            min_df=0.05, max_df=0.85)


    ftr_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
    kmeans = KMeans(n_clusters=clusternum, max_iter=10000, random_state=42)
    cluster_label = kmeans.fit_predict(ftr_vect)
    
    # 군집화한 레이블값들을 document_df 에 추가하기
    document_df['cluster_label'] = cluster_label
    return document_df.sort_values(by=['cluster_label'])
    

def get_topic_sentences(df, clabel):
    lst = []
    for i,t in enumerate(df[df['cluster_label'] == clabel]['opinion_text']):
        print(i, t)
        lst.append(t)
    return lst 


def delete_similar(input_sentences):
    if(len(input_sentences) < 2):
        return input_sentences
    sorted_sentences = sorted(input_sentences, key=lambda x:x[1][::-1])
    prev_len = len(sorted_sentences)
    
    for i in range(5):
        prev = sorted_sentences[0]
        processed_sentences = []
        for j,sentence in enumerate(sorted_sentences[1:]):
            s1 = set(prev[1].split())
            s2 = set(sentence[1].split())
            actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
            if(actual_jaccard < 0.5): # if not similar
                processed_sentences.append(prev)
                prev = sentence
            else:
                print(prev)
                print(sentence)
                print(actual_jaccard)
                print('-------------------------------------------')
                
        s1 = set(prev[1].split())
        
        if(len(processed_sentences) == 0):
            processed_sentences.append(prev)
            return processed_sentences
        
        s2 = set(processed_sentences[-1][1].split())
        actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
        if(actual_jaccard < 0.5): # if not similar
            processed_sentences.append(prev)
        
        sorted_sentences = sorted(processed_sentences, key=lambda x:x[1])
        print(prev_len, len(sorted_sentences))
        
        if(prev_len == len(sorted_sentences)):
            break
        prev_len = len(sorted_sentences)
        
    return sorted_sentences


def get_first_topk_sentences(df, topk_percent):
    first_sentences = []
    topk_sentences = []
    for a,b in zip(df['context'], df['topk']):
        context = eval(str(a))
        topk = eval(str(b))
        k = int(len(topk)*(topk_percent/100))
        topk = topk[:k]
        
        first = []
        for item in topk:
            if(item[0] == 0): 
                first.append(item)
                
        if(len(first) == 0):
            first_sentences += [(0, context[0])]
            topk_sentences += topk
        else:
            first_sentences += first
            topk.remove(first[0])
            topk_sentences += topk
                
    print('before delete similar:', len(first_sentences), len(topk_sentences))
    first_sentences = delete_similar(first_sentences)
    topk_sentences = delete_similar(topk_sentences)
    print('after delete similar:', len(first_sentences), len(topk_sentences))
    return first_sentences, topk_sentences


def get_additional_topk_sentences(df, prev_topk_percent, next_topk_percent):
    topk_sentences = []
    next_topk_percent = prev_topk_percent + next_topk_percent
    for a,b in zip(df['context'], df['topk']):
        context = eval(str(a))
        topk = eval(str(b))
        pk = int(len(topk)*(prev_topk_percent/100))
        k = int(len(topk)*(next_topk_percent/100))
        topk = topk[pk:k]
        topk_sentences += topk
                
    print('before delete similar:', len(topk_sentences))
    if(len(topk_sentences) == 0):
        return topk_sentences
    
    topk_sentences = delete_similar(topk_sentences)
    print('after delete similar:', len(topk_sentences))
    return topk_sentences    


def get_topk_sentences(k, user_input, model, tokenizer):
    bot_input_ids = News_to_input(user_input, openapi_key)
    
    chat_history_ids = summary(args, bot_input_ids, -1, '', None, model)
    pred_lst = list(chat_history_ids[0][:k])
    final_text = []
    for i,a in enumerate(user_input.split('.')):
        if i in pred_lst:
            #print(i, a)
            final_text.append(a+'. ')
    #print('------------------------------------------------------------')
    return final_text
    


def delete_repeat_str(user_text):
    text = user_text.split()
    x = len(text)
    comb = list(combinations(range(x), 2))
    sorted_comb = sorted(comb, key=lambda x: x[1]-x[0], reverse = True)
    for item in sorted_comb:
        start, end = item
        if(end-start <= len(sorted_comb)/2 and end-start>2):
            find_str = ' '.join(text[start:end])
            rest_str = ' '.join(text[end:])
            idx = rest_str.find(find_str)
            if idx != -1:
                print('deleted :', find_str)
                ret = ' '.join(text[:end]) + ' ' + rest_str[:idx] + rest_str[idx+len(find_str)+1:]
                return ret
    return user_text


def delete_repeat_in_list(sentences):
    new_final_sentences = []
    for s in sentences:
        prev = s
        new = delete_repeat_str(prev)
        while(new != prev):
            prev = new
            new = delete_repeat_str(prev)
            
        new_final_sentences.append(new)
        if(new != s):
            print(s)
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            print(new)
        print('---------------------------------')
    
    return new_final_sentences

        

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [178]:
topic_context_df = all_df[all_df.topic == 1]
len(topic_context_df)

3

In [179]:
def calc_len(sum_result2):
    ret = 0
    for s in sum_result2:
        tmp = 0
        for c in s[1]:
            tmp += len(c)
        ret += tmp
    return ret

In [180]:
import os
from os import listdir
from os.path import isfile, join
mypath = './bertopic(paraphrase)/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f.split('.')[1] == 'csv' and 'pre' in f.split('.')[0]]
onlyfiles

['bertopic_삼성전자_20221201_20221215_crwal_news_context_pre.csv',
 'bertopic_부동산_20221201_20221215_crwal_news_context_pre.csv',
 'bertopic_윤석열_20221201_20221215_crwal_news_context_pre.csv',
 'bertopic_삼성전자_20221201_20221203_crwal_news_context_pre.csv',
 'bertopic_윤석열_20221201_20221231_crwal_news_context_pre.csv',
 'bertopic_부동산_20221201_20221203_crwal_news_context_pre.csv',
 'bertopic_삼성전자_20221201_20221231_crwal_news_context_pre.csv',
 'bertopic_윤석열_20221201_20221203_crwal_news_context_pre.csv',
 'bertopic_부동산_20221201_20221231_crwal_news_context_pre.csv']

In [ ]:
import pandas as pd

for file in onlyfiles:
    all_df = pd.read_csv(f'./{mypath}/{name}.csv')
    print(all_df.topic.unique())
    
    for i in all_df.topic.unique():
        print('Topic:', i)
        topic_context_df = all_df[all_df.topic == i]

        first_sentences, topk_sentences = get_first_topk_sentences(topic_context_df, 15)

        sum_result1 = summarize_first_sentences(first_sentences, tokenizer, model)
        sum_result2 = sum_result1 + summarize_topk_sentences(topk_sentences, tokenizer, model)

        prev_topk = 15

        print('%%',calc_len(sum_result2),'$$')
        while(calc_len(sum_result2) < 450): # 너무 짧을 때
            print(f'................. get additional topk {prev_topk} ...............')
            topk_sentences2 = get_additional_topk_sentences(topic_context_df, prev_topk, 7)
            if(len(topk_sentences2) > 0):
                sum_result3 = summarize_topk_sentences(topk_sentences2, tokenizer, model)
                sum_result2 = sorted(sum_result3 + sum_result2, key=lambda x:x[0])
            prev_topk += 7

        final_result = ''
        for v in sum_result2:
            paragraph = [s.strip() for s in v[1]]
            paragraph = [s if('.' in s[-2: ]) else s+'. ' for s in paragraph]
            new_paragraph = delete_repeat_in_list(paragraph)
            final_result += ' '.join(new_paragraph)+'\n\n'


        print('======================= Final result =========================')
        print(final_result)
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        f = open(f'./{mypath}/text/{name}_summary_{i}.txt', 'w')
        f.write(final_result)
        f.close()

In [182]:
path = 'hdbscan'
name = 'hdbscan_윤석열_20221201_20221215_crwal_news_context_pre'
all_df = pd.read_csv(f'./{path}/{name}.csv')
all_df.topic.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [183]:
i = 10
print('Topic:', i)
topic_context_df = all_df[all_df.topic == i]
len(topic_context_df)

Topic: 10


11

In [185]:
first_sentences, topk_sentences = get_first_topk_sentences(topic_context_df, 15)

sum_result1 = summarize_first_sentences(first_sentences, tokenizer, model)
sum_result2 = sum_result1 + summarize_topk_sentences(topk_sentences, tokenizer, model)

prev_topk = 15

print('%%',calc_len(sum_result2),'$$')
while(calc_len(sum_result2) < 550): # 너무 짧을 때
    print(f'................. get additional topk {prev_topk} ...............')
    topk_sentences2 = get_additional_topk_sentences(topic_context_df, prev_topk, 7)
    if(len(topk_sentences2) > 0):
        sum_result3 = summarize_topk_sentences(topk_sentences2, tokenizer, model)
        sum_result2 = sorted(sum_result3 + sum_result2, key=lambda x:x[0])
    prev_topk += 7

final_result = ''
for v in sum_result2:
    paragraph = [s.strip() for s in v[1]]
    paragraph = [s if('.' in s[-2: ]) else s+'. ' for s in paragraph]
    new_paragraph = delete_repeat_in_list(paragraph)
    final_result += ' '.join(new_paragraph)+'\n\n'


print('======================= Final result =========================')
print(final_result)
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
f = open(f'./{path}/text/{name}_summary_{i}.txt', 'w')
f.write(final_result)
f.close()

before delete similar: 11 11
(0, '빌라 왕 피해구제 TF 구성 지시 윤석열 대통령이 15일 서울 청와대 영빈관에서 열린 제1차 국정과제 점검회의에 입장하고 있다.')
(0, '윤석열 대통령이 15일 서울 청와대 영빈관에서 열린 제1차 국정과제 점검회의를 주재하고 있다.')
0.5
-------------------------------------------
(0, '사진 확대 윤석열 대통령이 15일 서울 청와대 영빈관에서 열린 제1차 국정과제 점검회의를 주재하고 있다.')
(0, '제1차 국정과제 점검회의 주재 부동산, 정치 논리에 매몰 안 돼 윤석열 대통령이 15일 서울 청와대 영빈관에서 열린 제1차 국정과제 점검회의를 주재하고 있다.')
0.5454545454545454
-------------------------------------------
(0, '김진수 기자 디지털 뉴스 콘텐츠 이용규칙 보기 윤석열 대통령은 15일 다주택자에 대한 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 그런 여건을 만들어 드리려고 한 다 고 말했다.')
(0, '윤석열 대통령이 다주택자에 대한 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차를 할 수 있도록 그런 여건을 만들어 드리려고 한 다 고 밝혔다.')
0.5833333333333334
-------------------------------------------
11 8
8 8
11 11
after delete similar: 8 11
1
0 (0, '김진수 기자 디지털 뉴스 콘텐츠 이용규칙 보기 윤석열 대통령은 15일 다주택자에 대한 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 그런 여건을 만들어 드리려고 한 다 고 말했다.')
1 (0, '빌라 왕 피해구제 TF 구성 지시 윤석열 대통령이 15일 서울 청와대 영빈관에서 열린 제1차 국정과제 점검회의에 입장하고 있다.')
2 (0, '사진 확대 윤석열

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


1.3984 sec
윤석열 대통령은 15일 다주택자에 대한 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 여건을 만들어 드리려고 한 다 며 다주택자 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 그런 여건을 만들어 드리려고 한 다 고 밝혔다.
1 1
-----------------------------------------------
(0.0, ['윤석열 대통령은 15일 다주택자에 대한 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 여건을 만들어 드리려고 한 다 며 다주택자 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 그런 여건을 만들어 드리려고 한 다 고 밝혔다.'])
1
0 (7, ' 그는 많은 선의 의 피해자들이 지금 발을 동동 구르고 있는 그런 뉴스를 저도 계속 접했다 며 저도 법조인 출신이고 하다 보니까 이 부분에 대해 법률적으로 상당히 고민을 많이 하고 회의도 하고 했다 고  밝혔다')
1 (11, ' 민간과 공공 임대를 적절히 섞어 공급하겠다 고 말했다')
2 (3, ' 윤 대통령은 다주택자에 대한 세금을 완화하겠다고 하는 것도, 주택은 내가 사는 집 아니면 전부 임대를 놓게 돼 있기 때문에 다주택자에 중과세를 하게 되면 결국은 임대 물량에 대해서 비용이 많이 들어가기 때문에 영세 임차인에게  소위 세금의 전가가 일어난 다 고 설명했다')
3 (3, ' 윤 대통령은 제가 정부를 맡기 전까지는 공급과 수요 측면에 이런 불합리한 복합 규제 때문에 집값이 너무 천정부지로 솟고, 거래 물량이 위축됐다 며 전임 문재인 정부의 부동산 정책을 비판했다')
4 (9, ' 윤 대통령은 주택 가격과 관련해 집값이 오르고 내리는 문제는 기본적으로 시장 논리에 따라야 하는 것이지만, 정부는 그 완급을 잘 조절해서 예측 가능하게 관리하는 게 중요하다 면서 부동산 문제가 정치 논리나 이념에  매몰돼 선 안 된다 고 강조했다')
5 (9,

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


1.6432 sec
윤 대통령은 다주택자에 대한 세금을 완화하겠다고 하는 것도, 주택은 내가 사는 집 아니면 전부 임대를 놓게 돼 있기 때문에 다주택자에 대한 세금을 완화하겠다고 하는 것도, 주택은 내가 사는 집 아니면 전부 임대를 놓게 돼 있기 때문에 임대 물량에 대해서 비용이 많이 들어가기 때문에 영세 임차인에게  소위 세금의 전가가 일어난 다 고 말했다.
2 1
-----------------------------------------------
0.6994 sec
현재 다주택자는 규제지역에서는 주택담보대출을 받을 수 없지만 비규제지역에서는 주택담보 인정비율 LTV 60%까지 받을 수 있다
2 2
-----------------------------------------------
(7.818181818181818, ['윤 대통령은 다주택자에 대한 세금을 완화하겠다고 하는 것도, 주택은 내가 사는 집 아니면 전부 임대를 놓게 돼 있기 때문에 다주택자에 대한 세금을 완화하겠다고 하는 것도, 주택은 내가 사는 집 아니면 전부 임대를 놓게 돼 있기 때문에 임대 물량에 대해서 비용이 많이 들어가기 때문에 영세 임차인에게  소위 세금의 전가가 일어난 다 고 말했다.', '현재 다주택자는 규제지역에서는 주택담보대출을 받을 수 없지만 비규제지역에서는 주택담보 인정비율 LTV 60%까지 받을 수 있다'])
***************************************************
%% 423 $$
................. get additional topk 15 ...............
before delete similar: 5
5 5
after delete similar: 5
1
0 (5, ' 다주택자에 대한 과 한 세금이 결국 임대료 상승으로 이어진다고 보고 다주택자에 대한 세금을 깎아 임대료를 내리겠다는 것이다')
1 (6, ' 또 취득세 중과 폐지도 검토하고 있다')
2 (4, ' 윤 대통령은 공공임대주택에 대해 중앙 지방정부가 상

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


1.5978 sec
윤 대통령은 공공임대주택에 대해 중앙 지방정부가 상당한 재정 부담을 안으면서, 납세자에게 부담이 되고 경제 부담, 경기 위축 요인이 될 수 있다며, 중앙 지방정부가 상당한 재정 부담을 안으면서, 납세자에게 부담이 되고 경제 부담, 경기 위축 요인이 될 수 있다며, 공공주택에서 정부는 시행 이익을 포기하고 값 싸게 공급하면서 금융지원을 보조하겠다고 밝혔다.
1 1
-----------------------------------------------
(5.8, ['윤 대통령은 공공임대주택에 대해 중앙 지방정부가 상당한 재정 부담을 안으면서, 납세자에게 부담이 되고 경제 부담, 경기 위축 요인이 될 수 있다며, 중앙 지방정부가 상당한 재정 부담을 안으면서, 납세자에게 부담이 되고 경제 부담, 경기 위축 요인이 될 수 있다며, 공공주택에서 정부는 시행 이익을 포기하고 값 싸게 공급하면서 금융지원을 보조하겠다고 밝혔다.'])
***************************************************
deleted : 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록
deleted : 여건을 만들어 드리려고 한 다
윤석열 대통령은 15일 다주택자에 대한 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 여건을 만들어 드리려고 한 다 며 다주택자 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 그런 여건을 만들어 드리려고 한 다 고 밝혔다.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
윤석열 대통령은 15일 다주택자에 대한 과세를 경감해 시장에서 열악한 지 위에 있는 임차인들이 저가에 임차할 수 있도록 여건을 만들어 드리려고 한 다 며 다주택자 그런 고 밝혔다.
---------------------------------
deleted : 중앙 지방정부가 상당한 재정 부담을 안으면서, 납세자에게 부담이 되고 경제 부담, 경기 

In [155]:
for i in all_df.topic.unique():
    print('Topic:', i)
    topic_context_df = all_df[all_df.topic == i]

    first_sentences, topk_sentences = get_first_topk_sentences(topic_context_df, 15)

    sum_result1 = summarize_first_sentences(first_sentences, tokenizer, model)
    sum_result2 = sum_result1 + summarize_topk_sentences(topk_sentences, tokenizer, model)
    
    prev_topk = 15
    
    print('%%',calc_len(sum_result2),'$$')
    while(calc_len(sum_result2) < 550): # 너무 짧을 때
        print(f'................. get additional topk {prev_topk} ...............')
        topk_sentences2 = get_additional_topk_sentences(topic_context_df, prev_topk, 7)
        if(len(topk_sentences2) > 0):
            sum_result3 = summarize_topk_sentences(topk_sentences2, tokenizer, model)
            sum_result2 = sorted(sum_result3 + sum_result2, key=lambda x:x[0])
        prev_topk += 7

    final_result = ''
    for v in sum_result2:
        paragraph = [s.strip() for s in v[1]]
        paragraph = [s if('.' in s[-2: ]) else s+'. ' for s in paragraph]
        new_paragraph = delete_repeat_in_list(paragraph)
        final_result += ' '.join(new_paragraph)+'\n\n'


    print('======================= Final result =========================')
    print(final_result)
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    f = open(f'./bertopic(mpnet)/{name}_summary_{i}.txt', 'w')
    f.write(final_result)
    f.close()

Topic: 0
before delete similar: 6 10
6 6
10 10
after delete similar: 6 10
1
0 (0, '국내 금융시장이 경색 국면에서는 벗어났으나 CP 시장을 중심으로 한 불안감은 계속되고 있는 것으로 나타났다.')
1 (0, '금융 당국이 자금시장 경색의 진원지인 부동산 프로젝트파이낸싱 PF 자산유동화기업어음 ABCP 사업장 현황에 대한 모니터링을 강화해 나가기로 했다.')
2 (0, '남산에서 바라본 서울 아파트들 사진 연합뉴스 자금시장 경색의 진원지이자 금융시장의 취약 부문으로 손 꼽히는 부동산 프로젝트파이낸싱 PF 자산유동화기업어음 ABCP 시장에 대한 부실 우려가 계속되고 있다.')
3 (0, '내년 상반기 건설 업체들의 부도가 급증하고 하반기부터는 제2금융권 부실로 전이 돼 경제에 2차 충격이 가해질 가능성이 제기됐다.')
4 (0, '서울 강동구 둔촌주공아파트 재건축 현장의 모습.')
5 (0, '저금리 기조 하 금융기관 도덕적 해이 문제 지적 아시아경제 서소정 기자 한국은행은 레고랜드 발 사태로 촉발된 국내 기업어음 CP 신용채권 시장 불안이 정부의 시장 안정 대책 효과로 점차 진정되겠으나 국제금융시장의 높은 불확실성, 부동산 프로젝트파이낸싱 PF 부실화, 연말 자금 수급 악화 가능성 등 리스크 요인이 상존한 만큼 필요시 적절한 시장 안정화 대책을   강구할 필요가 있다고 판단했다.')


/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


1.7640 sec
자금시장 경색의 진원지이자 금융시장의 취약 부문으로 손 꼽히는 부동산 프로젝트파이낸싱 PF 자산유동화기업어음 ABCP 시장에 대한 부실 우려가 계속되고 있으며 금융당국은 자금시장 경색의 진원지인 부동산 프로젝트파이낸싱 PF 자산유동화기업어음 ABCP 사업장 현황에 대한 모니터링을 강화해 나가기로 했다.
1 1
-----------------------------------------------
(0.0, ['자금시장 경색의 진원지이자 금융시장의 취약 부문으로 손 꼽히는 부동산 프로젝트파이낸싱 PF 자산유동화기업어음 ABCP 시장에 대한 부실 우려가 계속되고 있으며 금융당국은 자금시장 경색의 진원지인 부동산 프로젝트파이낸싱 PF 자산유동화기업어음 ABCP 사업장 현황에 대한 모니터링을 강화해 나가기로 했다.'])
1
0 (7, ' 10월  중순 경에는 시장 불안이 국고채 투자심리에도 영향을 주면서 국고채 금리가 주요국 금리에 비해 변동성이 크게 확대되는 등 시장 전반으로 불안 심리가 확산됐다')
1 (8, ' 3년 만기 AA 등급 회사채 금리는 10월 %까지 올랐지만 이달 9일에는 %까지 하락했다')
2 (8, ' 공사채 은행채 발행 물량이 소화되고 발행 스프레드도 낮아졌지만 회사채 여전채 발행 부진과 증권사 CP 및 PF ABCP의 차환 어려움은 계속되고 있다')
3 (6, ' 다만 CP 시장을 중심으로 여전히 높은 신용 경계감이 지속되고 있다')
4 (15, ' 당국은 PF ABCP 매입 프로그램을 가동해 만기 도래에 따른 시장 수요에 맞춰 지원을 지속한다는 방침이지만 가격 하락이 진정되지 않을 경우 앞으로 PF ABCP 상환은 더 어려워질 수 있다는 지적이 나온 다')
5 (5, ' 부동산 개발사업과 관련한 대출채권이 기초자산이다')
6 (6, ' 시행사의  경우 신용도가 낮아 시공사와 금융사가 신용을 보강하거나 유동성을 지원하는 식으로 PF에 참여한다')
7 (8, ' 이어 분양 전망은 좋지 않지만 임대  수요는 급증하고 있는 만큼 건

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.8737 sec
10월 중순 중순 경에는 시장 불안이 국고채 투자심리에도 영향을 주면서 국고채 금리가 주요국 금리에 비해 변동성이 크게 확대되는 등 시장 전반으로 불안 심리가 확산됐다.
2 1
-----------------------------------------------
0.4383 sec
부동산 가격이 하락세로 전화하면서 상환 및 차환 우려가 확대되고 있다
2 2
-----------------------------------------------
(8.1, ['10월 중순 중순 경에는 시장 불안이 국고채 투자심리에도 영향을 주면서 국고채 금리가 주요국 금리에 비해 변동성이 크게 확대되는 등 시장 전반으로 불안 심리가 확산됐다.', '부동산 가격이 하락세로 전화하면서 상환 및 차환 우려가 확대되고 있다'])
***************************************************
%% 301 $$
................. get additional topk 15 ...............
before delete similar: 4
4 4
after delete similar: 4
1
0 (10, ' 국제 금융시장의 불확실성이 높 고 부동산 PF 부실화 가능성도 남아 있다')
1 (7, ' 비수도권 사업장을 중심으로 사업성 악화와 미분양이 확산하면서 자금조달 어려움 및 부실 확대 가능성이 커진 탓이다')
2 (14, ' 실제로 10 11월 차환 과정에서 만기 구조가 3개월 단위에서 1 2개월 등으로 단기화하는 사례가 나타나 증권사 자산건전성 변동이 커지고 있다')
3 (5, ' 이 때문에 사업이 제대로  진척되지 않을 경우 신용공여를 한 증권사의 건전성까지 훼손될 가능성이 있다')
---------------------------------------------------


...

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


5.3071 sec
내년 1월1일부터 증여 취득세 과세 표준이 시가 표준액 에서 시가 인정액 으로 바뀐 가운데, 미국의 주택 투자는 경제에서 차지하는 비중이 4%에 불과하지만 급격한 주택경기 위축은 3분기에 큰 영향을 미쳐 총 성장률을 %나 마이너스로 차감시켰고, 연준, 기준금리 %p 예측 한은과 %차 벌어져 악영향 경인 매매 수급지수 모두 60대 국토교통부가 발표한 10월 주택 통계 자료를 보면 수도권 주택 누적 거래량은 지난 10월까지 17만 9천159건으로 전년 동기 대비 % 감소했다 
이번 미국의 깜짝 국내총생산 GDP 수치 이면에는 다른 이야기가 숨겨져 있다 
이번 미국의 깜짝 국내총생산 GDP 수치 이면에는 다른 이야기가 숨겨져 있다 
이번 미국의 깜짝 국내총생산 GDP 수치 이면에는 다른 이야기가 숨겨져 있다 
이번 미국의 깜짝 국내총생산 GDP 수치 이면에는 다른 이야기가 숨겨져 있다 
이번 미국의 깜짝 국내총생산 GDP 수치 이면에는 다른 이야기가 숨겨져 있다 
이번 미국의 깜짝 국내총생산 GDP 수치 이면에는 다른 이야기가 숨겨져 있다 
이번 미국의 깜짝 국내총생산 GDP 수치 이면에는 다른 이야기가 숨겨져 있다 
이번 미국의 깜짝 국내총생산 GDP 수치
1 1
-----------------------------------------------
(10.2, ['내년 1월1일부터 증여 취득세 과세 표준이 시가 표준액 에서 시가 인정액 으로 바뀐 가운데, 미국의 주택 투자는 경제에서 차지하는 비중이 4%에 불과하지만 급격한 주택경기 위축은 3분기에 큰 영향을 미쳐 총 성장률을 %나 마이너스로 차감시켰고, 연준, 기준금리 %p 예측 한은과 %차 벌어져 악영향 경인 매매 수급지수 모두 60대 국토교통부가 발표한 10월 주택 통계 자료를 보면 수도권 주택 누적 거래량은 지난 10월까지 17만 9천159건으로 전년 동기 대비 % 감소했다 \n이번 미국의 깜짝 국내총생산 GDP 수치 이면에는 다른 이야기가 숨겨져 있다 \n이번 미국의 깜짝 국내총생산 G

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


1.6173 sec
중소형 추첨제 비율은 투기과열지구 0%, 조정대상지역 25%여서 가점이 낮은 청년가구의 당첨이 어려웠던 중소형 추첨제 비율은 투기과열지구 0%, 조정대상지역 25%여서 가점이 낮은 청년가구의 당첨이 어려웠다.
1 1
-----------------------------------------------
(9.8, ['중소형 추첨제 비율은 투기과열지구 0%, 조정대상지역 25%여서 가점이 낮은 청년가구의 당첨이 어려웠던 중소형 추첨제 비율은 투기과열지구 0%, 조정대상지역 25%여서 가점이 낮은 청년가구의 당첨이 어려웠다.'])
***************************************************
%% 230 $$
................. get additional topk 15 ...............
before delete similar: 3
3 3
after delete similar: 3
1
0 (6, ' 이에 따라 그동안 주택도시기금에서 만 34세 이하 청년을 대상으로 시행하는 버팀 목 전세대출을 이용하지 못했던 청년층들의 수요가 많을 것으로 예상한다')
1 (6, ' 이에 추첨제 비율을 전용면적 60 이 하는 60%, 60 85 는 30%로 늘렸다')
2 (7, ' 이와 함께 중장년 가구 수요가 많은 전용 85 초과에 대해서는 가점제 물량이 확대된다')
---------------------------------------------------
(6.333333333333333, [' 이에 따라 그동안 주택도시기금에서 만 34세 이하 청년을 대상으로 시행하는 버팀 목 전세대출을 이용하지 못했던 청년층들의 수요가 많을 것으로 예상한다'])
***************************************************
................. get additional topk 22 ...............
before delete similar: 3
3 3
after dele

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is no

In [ ]:
f = open('./bertopic(mpnet)/bertopic_부동산_20221201_20221203_crwal_news_context_summary.txt', 'w')
f.write(final_result)
f.close()